In [1]:
run accelerator.py

RadioButtons(description='Device:', options=('GPU Gen9', 'GPU Iris XE Max', 'CPU Xeon 6128', 'CPU Xeon 8153'),…

##Dining Philosophers problem in SYCL
 The given code is a basic SYCL implementation to simulate the Dining Philosophers problem. This classic synchronization problem involves philosophers sitting around a table, with each philosopher having a chopstick between them. They can either think or eat; to eat, a philosopher needs both chopsticks adjacent to them. The goal is to find a solution that avoids deadlock and starvation. The code models this problem as follows:

The number of philosophers (NUM_PHILOSOPHERS) is set to 5, and the number of iterations (NUM_ITERATIONS) is set to 5.

A vector chopsticks is created to represent the chopsticks, with each element being an integer (1 = available, 0 = in use). A buffer chopsticks_buf is created from this vector.

The kernel dining_philosophers is launched with one work-item per philosopher. Inside the kernel, each philosopher tries to pick up their left and right chopsticks atomically. Once they have both, they "eat" and then "think". The code simulates the Dining Philosophers problem as follows:

a. Determine the left and right chopstick indices for each philosopher.

b. Loop NUM_ITERATIONS times for each philosopher:

i. Try to pick up the left chopstick by using an atomic fetch_or operation, and busy-wait if it's not available.

ii. Try to pick up the right chopstick using an atomic fetch_or operation, and busy-wait if it's not available.

iii. Print that the philosopher is eating.

iv. Release the left and right chopsticks by setting their values to 1.

v. Print that the philosopher is thinking.









In [2]:
%%writefile lab/dphi2.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <vector>
#include <thread>
#include <chrono>
#include <atomic>
#include <algorithm>

constexpr size_t num_philosophers = 5;

std::atomic<bool> done(false);

void philosopher(size_t id, std::vector<std::atomic<bool>>& forks) {
    size_t left_fork = id;
    size_t right_fork = (id + 1) % num_philosophers;

    while (!done) {
        // Try to pick up left fork
        bool expected = false;
        while (!forks[left_fork].compare_exchange_weak(expected, true)) {
            expected = false;
            std::this_thread::yield();
        }

        // Try to pick up right fork
        expected = false;
        while (!forks[right_fork].compare_exchange_weak(expected, true)) {
            expected = false;
            std::this_thread::yield();
        }

        // Eating
        std::cout << "Philosopher " << id << " is eating." << std::endl;
        std::this_thread::sleep_for(std::chrono::milliseconds(100));

        // Put down forks
        forks[left_fork] = false;
        forks[right_fork] = false;

        // Thinking
        std::this_thread::sleep_for(std::chrono::milliseconds(100));
    }
}

int main() {
    std::vector<std::atomic<bool>> forks(num_philosophers);
    std::fill(forks.begin(), forks.end(), false);

    std::vector<std::thread> philosophers(num_philosophers);
    for (size_t i = 0; i < num_philosophers; ++i) {
        philosophers[i] = std::thread(philosopher, i, std::ref(forks));
    }

    std::this_thread::sleep_for(std::chrono::seconds(5));
    done = true;

    for (auto& philosopher_thread : philosophers) {
        philosopher_thread.join();
    }

    return 0;
}


Writing lab/dphi2.cpp


In [3]:
! chmod 755 q; chmod 755 run_simple03.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple03.sh; else ./run_simple03.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Selected Device is: 

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2283766.v-qsvr-1           ...ub-singleuser u182761         00:00:13 R jupyterhub     
2283787.v-qsvr-1           run_simple03.sh  u182761                0 Q batch          

Waiting for Output █████████████████████████████████████████████ Done⬇

########################################################################
#      Date:           Mon 24 Apr 2023 03:51:31 AM PDT
#    Job ID:           2283787.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gen9:ppn=2,nodes=1:gen9:ppn=2,walltime=06:00:00
########################################################################

## u182761 is compiling SYCL Code -- oneAPI Hack2Skill example - 1 of 1 dphi.cpp
Running on device: Intel(R) UHD Graphics P630 [0x